In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
import os
import pickle
import concurrent.futures
sys.path.append("..") 

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm_notebook as tqdm
from nlp_surveillance.scraper import who_scraper, promed_scraper, text_extractor
from nlp_surveillance import my_utils

In [6]:
if not my_utils.connection_is_possible():
    my_utils.assure_right_proxy_settings()
    headers = my_utils.load_rki_header_and_proxy_dict()['headers']
    proxy = my_utils.load_rki_header_and_proxy_dict()['proxy']
    proxy = dict(zip(["http", "https"], proxy.values()))

In [ ]:
who_urls = who_scraper.scrape(proxy=proxy)

In [ ]:
who_urls = who_urls.URL.tolist()

In [ ]:
path = os.path.join("..", "data", "corpus", "who_urls.pkl")
with open(path, 'wb') as f:
    pickle.dump(who_urls, f)

In [18]:
with open('../data/corpus/who_urls.pkl', 'rb') as f:
    who_urls = pickle.load(f)

In [ ]:
len(who_urls)

In [11]:
promed_urls = promed_scraper.scrape(year_range=(1994, 2018), proxy=proxy)

In [12]:
promed_urls = promed_urls.URL.tolist()

In [14]:
path = os.path.join("..", "data", "corpus", "promed_urls.pkl")
with open(path, 'wb') as f:
    pickle.dump(promed_urls, f)

In [15]:
len(promed_urls)

54959

In [19]:
all_urls = promed_urls
all_urls.extend(who_urls)

In [33]:
for url in tqdm(all_urls):
    text = text_extractor.extract_cleaned_text_from_url(url, proxy)
    url_formatted = url.replace("://", "_").replace("/", "_")
    path_name = os.path.join("..", "data", "corpus", f"{url_formatted}.txt")
        
    with open(path_name, 'w', encoding="utf-8") as f:
        f.write(text) 
